In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "CS340Crevier"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################

# Setup the Dash application
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo (Ensure the logo file is available in your working directory)
image_filename = 'Grazioso Salvare Logo.png'  # Update to your image file name
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Display the logo and unique identifier at the top of the dashboard
    html.Center([
        html.A(
            href='https://www.snhu.edu',  # Link to client's homepage
            children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '20%', 'height': 'auto'})
        ),
        html.B(html.H1("Michael Crevier's Unique Dashboard"))
    ]),
    html.Hr(),
    # Div for filtering options
    html.Div([
        # Adding radio buttons for filtering rescue types
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'ALL'}
            ],
            value='ALL',
            labelStyle={'display': 'inline-block'}
        ),
        html.Br(),
        # Adding a dropdown to select the number of rows to display
        html.Div([
            html.Label("Select number of rows to display:"),
            dcc.Dropdown(
                id='rows-per-page',
                options=[
                    {'label': '5', 'value': 5},
                    {'label': '10', 'value': 10},
                    {'label': '25', 'value': 25},
                    {'label': '100', 'value': 100},
                ],
                value=5,  # Default value
                style={'width': '100px', 'height': '30px', 'display': 'inline-block', 'vertical-align': 'middle'}
            )
        ], style={'display': 'inline-block', 'margin-left': '20px'})
    ]),
    html.Hr(),
    # DataTable setup
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         row_selectable="single",
                         selected_rows=[0],
                         page_action="native",
                         page_size=5,  # Number of rows per page
                         style_table={'overflowX': 'auto'},
                        ),
    html.Br(),
    html.Hr(),
    # Set up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display': 'flex'},
         children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%'}
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%'}
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################



    
# Callback for updating the data table based on the selected filter and rows per page
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'page_size')],
    [Input('filter-type', 'value'),
     Input('rows-per-page', 'value')]
)
def update_dashboard(filter_type, rows_per_page):
    # Filter the data based on the rescue type
    if filter_type == 'ALL':
        filtered_df = df
    elif filter_type == 'Water Rescue':
        filtered_df = df[
            (df['breed'].str.contains('Labrador Retriever|Chesapeake Bay Retriever|Newfoundland', case=False)) &
            (df['sex_upon_outcome'].str.contains('Female', case=False)) &
            (df['age_upon_outcome_in_weeks'] >= 26) & 
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif filter_type == 'Mountain Rescue':
        filtered_df = df[
            (df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler', case=False)) &
            (df['sex_upon_outcome'].str.contains('Male', case=False)) &
            (df['age_upon_outcome_in_weeks'] >= 26) & 
            (df['age_upon_outcome_in_weeks'] <= 156)
        ]
    elif filter_type == 'Disaster Rescue':
        filtered_df = df[
            (df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False)) &
            (df['sex_upon_outcome'].str.contains('Male', case=False)) &
            (df['age_upon_outcome_in_weeks'] >= 20) & 
            (df['age_upon_outcome_in_weeks'] <= 300)
        ]
    
    return filtered_df.to_dict('records'), rows_per_page

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [dcc.Graph(
            figure={
                "data": [],
                "layout": {"title": "No Data Available"}
            }
        )]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if 'breed' column exists before creating the pie chart
    if 'breed' not in dff.columns:
        return [dcc.Graph(
            figure={
                "data": [],
                "layout": {"title": "No Data Available"}
            }
        )]
    
    # Calculate the percentage of each breed
    breed_counts = dff['breed'].value_counts(normalize=True) * 100
    
    # Separate out breeds with less than 2% into 'All Other Breeds'
    other_breeds = breed_counts[breed_counts < 2].index
    dff['breed'] = dff['breed'].apply(lambda x: 'All Other Breeds' if x in other_breeds else x)
    
    # Recalculate the breed counts to include 'All Other Breeds'
    final_counts = dff['breed'].value_counts()
    
    # Create a pie chart based on the breed of the animals with the legend below
    fig = px.pie(final_counts, names=final_counts.index, values=final_counts.values, title='Preferred Animals by Breed')
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-1.5,  # Adjust this value to move the legend further below the chart
            xanchor="center",
            x=0.5
        )
    )
    
    return [
        dcc.Graph(
            figure=fig
        )
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Handle case where DataFrame is empty or no row is selected
    if dff.empty or index is None or len(index) == 0:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])]
    
    row = index[0]
        
    # Safely retrieve latitude and longitude using column names
    lat = dff.iloc[row]['location_lat'] if 'location_lat' in dff.columns else 30.75
    lon = dff.iloc[row]['location_long'] if 'location_long' in dff.columns else -97.48
    
        # Safely retrieve animal name and breed using column names
    animal_name = dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown"
    animal_breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"
    
    # Check if latitude and longitude are valid, if not, use default coordinates
    if pd.isnull(lat) or pd.isnull(lon):
        lat, lon = 30.75, -97.48
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(f"Name: {animal_name}"),
                    html.P(f"Breed: {animal_breed}")
                ])
            ])
        ])
    ]

# Run the app and display the result
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10372/
